In [11]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR

In [12]:
df = pd.read_csv('gurgaon_properties_post_feature_selection.csv')

In [13]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0.0,40.0,3,2,2.0,1.0,850.0,0,0,0,1.0,1.0,0.82
1,0.0,101.0,2,2,2.0,1.0,1226.0,1,0,0,1.0,2.0,0.95
2,0.0,111.0,2,2,1.0,1.0,1000.0,0,0,0,1.0,0.0,0.32
3,0.0,105.0,3,4,4.0,3.0,1615.0,1,0,1,0.0,2.0,1.60
4,0.0,4.0,2,2,1.0,3.0,582.0,0,1,0,0.0,2.0,0.48


In [14]:
X = df.drop(columns=['price'])
y = df['price']

In [15]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [16]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [ ]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(),['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first'), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [17]:
# Example: Replace 'numeric_cols' and 'categorical_cols' with your actual column names
numeric_cols = ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']
categorical_cols = ['categorical_column1', 'categorical_column2']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(), columns_to_encode)
    ]
)


In [18]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [28]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [29]:
y_transformed

0       0.598837
1       0.667829
2       0.277632
3       0.955511
4       0.392042
          ...   
3582    0.314811
3583    1.945910
3584    0.470004
3585    2.803360
3586    1.022451
Name: price, Length: 3587, dtype: float64

In [30]:
X.mean()

property_type         0.218288
sector               60.991915
bedRoom               3.203791
bathroom              3.272930
balcony               2.718706
agePossession         2.112350
built_up_area      1850.484003
servant room          0.355729
store room            0.089490
furnishing_type       0.388904
luxury_category       1.270700
floor_category        1.281015
dtype: float64

In [31]:
scores.mean()

nan